In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
# Test with df1 with 5 rows

In [12]:
df1 = pd.read_csv('output_file.csv')

In [13]:
df1

,latitude,longitude
0,41.82,-72.31
1,48.45,-122.61
2,33.95,-118.41
3,31.49,-98.52
4,41.02,-72.31
...,...,...
140,40.15,-105.12
141,42.05,-123.11
142,29.58,-98.93
143,42.48,-70.89


In [5]:
#Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

#Make sure all required weather variables are listed here
#The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive" 

# Create an empty list to store the responses
all_responses = []

# Iterate through all rows in df1
for index, row in df1.iterrows():
    params = {
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "start_date": "2010-01-01",
        "end_date": "2011-12-31",
        "daily": ["temperature_2m_mean", "daylight_duration", "sunshine_duration", "rain_sum", "snowfall_sum"]
    }

    # Make the Open-Meteo API request and append the response to the list
    response = openmeteo.weather_api(url, params=params)
    all_responses.append(response)

# Display the list of responses
print(all_responses)

[[<openmeteo_sdk.WeatherApiResponse.WeatherApiResponse object at 0x0000021DB3F3A6B0>], [<openmeteo_sdk.WeatherApiResponse.WeatherApiResponse object at 0x0000021DB3F399F0>], [<openmeteo_sdk.WeatherApiResponse.WeatherApiResponse object at 0x0000021DB3EFFE50>], [<openmeteo_sdk.WeatherApiResponse.WeatherApiResponse object at 0x0000021DB3F3B5E0>], [<openmeteo_sdk.WeatherApiResponse.WeatherApiResponse object at 0x0000021DB3F3B0D0>]]


In [6]:
# Initialize an empty DataFrame to store all daily data
all_daily_data = pd.DataFrame()

# Process each location's response in a loop
for responses in all_responses:
    # Iterate over each response in the list
    for response in responses:
        print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
        print(f"Elevation {response.Elevation()} m asl")
        print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
        print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

        # Process daily data
        daily = response.Daily()
        daily_daylight_duration = daily.Variables(0).ValuesAsNumpy()
        daily_sunshine_duration = daily.Variables(1).ValuesAsNumpy()
        daily_rain_sum = daily.Variables(2).ValuesAsNumpy()
        daily_snowfall_sum = daily.Variables(3).ValuesAsNumpy()

        daily_data = {
            "date": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit="s"),
                end=pd.to_datetime(daily.TimeEnd(), unit="s"),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            ),
            "daylight_duration": daily_daylight_duration,
            "sunshine_duration": daily_sunshine_duration,
            "rain_sum": daily_rain_sum,
            "snowfall_sum": daily_snowfall_sum
        }

        daily_dataframe = pd.DataFrame(data=daily_data)
        
        # Append data for the current location to the overall DataFrame
        all_daily_data = pd.concat([all_daily_data, daily_dataframe], ignore_index=True)

# Display the resulting DataFrame for all locations
print(all_daily_data)

Coordinates 41.79261779785156°E -72.2608642578125°N
Elevation 132.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 41.79261779785156°E -72.2608642578125°N
Elevation 132.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 41.79261779785156°E -72.2608642578125°N
Elevation 132.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 41.79261779785156°E -72.2608642578125°N
Elevation 132.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 41.79261779785156°E -72.2608642578125°N
Elevation 132.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
           date  daylight_duration  sunshine_duration      rain_sum  \
0    2010-01-01          -1.296000       33137.511719  21664.470703   
1    2010-01-02          -2.312667       33183.515625    401.293030   
2    2010-01-03          -9.658501       33233.257812   9730.592773   
3    2010-01-04          -5.791833       33286.687500  29159.111328   
4    2010-01-05

In [9]:
all_daily_data

,date,daylight_duration,sunshine_duration,rain_sum,snowfall_sum
0,2010-01-01,-1.296000,33137.511719,21664.470703,0.0
1,2010-01-02,-2.312667,33183.515625,401.293030,0.0
2,2010-01-03,-9.658501,33233.257812,9730.592773,0.0
3,2010-01-04,-5.791833,33286.687500,29159.111328,0.0
4,2010-01-05,-5.825167,33343.757812,29045.250000,0.0
...,...,...,...,...,...
3645,2011-12-27,1.501917,32946.882812,6383.548340,1.4
3646,2011-12-28,6.841501,32971.207031,28385.228516,13.9
3647,2011-12-29,-2.554333,33000.082031,28951.291016,0.0
3648,2011-12-30,0.710250,33033.421875,28594.076172,0.0


In [10]:
latitudes = []
longitudes = []

# Process each location's response in a loop
for responses in all_responses:
    # Iterate over each response in the list
    for response in responses:
        # Extract latitude and longitude
        latitude = response.Latitude()
        longitude = response.Longitude()

        # Append latitude and longitude to the lists
        latitudes.append(latitude)
        longitudes.append(longitude)


In [11]:
# Add latitude and longitude columns to the DataFrame
all_daily_data['latitude'] = latitudes
all_daily_data['longitude'] = longitudes

ValueError: Length of values (5) does not match length of index (3650)

In [ ]:
all_daily_data.head()

In [ ]:
# Code if I only use unique dates


#Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

# Create an empty list to store the responses
all_responses = []

# Iterate through all unique dates
for Date in unique_dates_df['Date']:
    # Get the corresponding rows for the current date
    date_rows = merged_df[merged_df['Date'] == Date]

    # Use the latitude and longitude values from the first row (assuming they are the same for all rows with the same date)
    latitude = date_rows.iloc[0]['latitude']
    longitude = date_rows.iloc[0]['longitude']

    # Format the date in 'YYYY-MM-DD' format
    formatted_date = Date.strftime('%Y-%m-%d')

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": formatted_date,
        "end_date": formatted_date,
        "daily": ["temperature_2m_mean", "daylight_duration", "sunshine_duration", "rain_sum", "snowfall_sum"]
    }

    # Make the Open-Meteo API request and append the response to the list
    response = openmeteo.weather_api(url, params=params)
    all_responses.append(response)

# Display the list of responses
print(all_responses)